In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Relaxtion ##

A simple code "example" of the relaxation method.
Solves $d^2\Phi/dx^2 = 0$ with $\Phi(0)=1$ and $\Phi(1)=0$ using what is possibly the simplest iteration scheme.

Discretize the spatial coordinate ($x$) on a regular grid: $x_i=ih$ with $i$ ($0\le i\le N$) an integer, $h=1/N$ and $N+1$ the number of grid points.
We start with the 1D version of
$$
  \nabla^2\Phi = f \quad{\rm with}\quad
  \nabla^2\Phi \approx \frac{\Phi(x+h)-2\Phi(x)+\Phi(x-h)}{h^2}
$$
with $f=-4\pi\rho=0$ for all interior points $0<i<N$.  By rewriting the Poisson equation as
$$
  \Phi_i = \frac{1}{2}\left( \Phi_{i+1} + \Phi_{i-1} - h^2f_i \right)
$$
we get an iterative method for solution.

In [ ]:
def solve(phi0=1,phi1=0,n=10,Niter=50,verbose=False):
    """A not-very-optimized but (hopefully) easy to read relaxation code."""
    hh      = 1.0/float(n)
    xi      = np.linspace(0,1,n+1)
    ff      = np.zeros(n+1)
    phi     = np.zeros(n+1)
    phi[ 0] = phi0
    phi[-1] = phi1
    #
    for iter in range(Niter):
        phi_old = phi
        for i in range(1,n):
          phi[i] = 0.5*(phi_old[i-1]+phi_old[i+1]-hh**2*ff[i])
        if verbose:
            print(phi)
    #
    exact = 1-xi
    return( (phi,exact) )
    #

In [ ]:
phi,exact = solve(Niter=25)
for i in range(phi.size):
        print("{:3d} {:10.5f} {:10.5f} {:10.3f}".format(i,phi[i],exact[i],\
          100.*(phi[i]-exact[i])/(exact[i]+1e-10)))

Let's look at the error as a function of the number of iterations, for 10 grid points.

In [ ]:
n=10
fig,ax = plt.subplots(2,1,figsize=(10,7),sharex=True)
phi,exact = solve(n=n,Niter=1)
xx = np.linspace(0.,1.,phi.size,endpoint=True)
ax[0].plot(xx,exact,'k--',label='Exact')
for Niter in [10,20,50]:
    phi,exact = solve(n=n,Niter=Niter)
    ax[0].plot(xx,phi,':',label="Relax($N={:3d}$)".format(Niter))
    ax[1].plot(xx[:-1],(100.*(phi-exact)/(exact+1e-10))[:-1],'-')
ax[0].legend()
ax[0].set_ylabel(r'$\Phi(x)$')
ax[1].set_ylabel(r'Error (%)')
ax[1].set_xlabel(r'$x$')

Now let's look at how the rate of convergence slows down as we increase the number of grid points.

In [ ]:
n=50
fig,ax = plt.subplots(2,1,figsize=(10,7),sharex=True)
phi,exact = solve(n=n,Niter=1)
xx = np.linspace(0.,1.,phi.size,endpoint=True)
ax[0].plot(xx,exact,'k--',label='Exact')
for Niter in [250,500,1000]:
    phi,exact = solve(n=n,Niter=Niter)
    ax[0].plot(xx,phi,':',label="Relax($N={:3d}$)".format(Niter))
    ax[1].plot(xx[:-1],(100.*(phi-exact)/(exact+1e-10))[:-1],'-')
ax[0].legend()
ax[0].set_ylabel(r'$\Phi(x)$')
ax[1].set_ylabel(r'Error (%)')
ax[1].set_xlabel(r'$x$')

One can study the rate of convergence analytically, and then devise methods to accelerate the convergence of the modes which converge the slowest -- this leads to "multigrid", which is one of the most useful techniques for solving PDEs on a grid (especially in the era of massively parallel computing since one can frequently efficiently interleave communication and computation).

A useful introduction to multigrid methods is the slim volume ["A multigrid tutorial" by Briggs, et al.](https://www.amazon.com/Multigrid-Tutorial-William-L-Briggs/dp/0898714621).